# Programación declarativa @ GIA - URJC
## Curso 23-24
## Convocatoria extraordinaria
## Prueba 2

La duración de la prueba es de 1 hora y 20 minutos.


# Definiciones auxiliares

In [ ]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

In [ ]:
object Signatures:
    abstract class List[A]:
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def reduce(f: (A, A) => A): A
 
        // Common functions
        def length: Int
        def reverse: List[A]
        def flatten[B](/* A debe poder convertirse al tipo List[B] */): List[B]

        // devuelve la lista formada por los `i` primeros elementos
        def take(i: Int): List[A]

        // devuelve la lista resultante de eliminar los `i` primeros elementos
        def drop(i: Int): List[A]

    abstract class StringOps(s: String): 

        // Devuelve la primera posición de la cadena de caracteres en la que el predicado es cierto para el carácter en dicha posición.
        // Si no existe ningún carácter que satisfaga el predicado, la función devuelve el valor -1.
        def indexWhere(pred: Char => Boolean): Int 

        // Particiona el string en dos substrings de acuerdo a un índice `i`, el primero de ellos con los caracteres desde la posición 0..i-1
        // y el segundo con los caracteres de las posiciones i..length
        def splitAt(i: Int): (String, String)

        // Elimina `i` caracteres del comienzo del string. 
        def drop(i: Int): String

In [ ]:
// type Tree[A] = 1 + Tree[A] * A * Tree[A]

enum Tree[A]:
    case Empty()
    case Node(left: Tree[A], root: A, right: Tree[A])

object Tree:
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)

import Tree._

# Ejercicio 1
__(4 puntos)__

Se desea implementar una función `cut` que permita particionar una lista de elementos en sublistas contiguas de un tamaño dado (excepto la última sublista, que podrá ser de un tamaño inferior si la longitud de la lista no es múltiplo de dicho tamaño). El siguiente test ilustra el comportamiento de la función para elementos de tipo `Int`:

In [ ]:
class TestCut(cut: Int => List[Int] => Option[List[List[Int]]]) extends AnyFlatSpec with should.Matchers:

    "cut" should "work" in:
        cut(0)(List(1,2,3,4)) shouldBe None
        cut(1)(List(1,2,3,4,5)) shouldBe Some(List(List(1), List(2), List(3), List(4), List(5)))
        cut(2)(List()) shouldBe Some(List())
        cut(2)(List(1)) shouldBe Some(List(List(1)))
        cut(2)(List(1,2,3,4)) shouldBe Some(List(List(1,2),List(3,4)))
        cut(2)(List(1,2,3,4,5)) shouldBe Some(List(List(1,2),List(3,4),List(5)))
        cut(3)(List(1,2,3,4,5)) shouldBe Some(List(List(1,2,3),List(4,5)))
        cut(6)(List(1,2,3,4,5)) shouldBe Some(List(List(1,2,3,4,5)))

Obsérvese que la función solo devuelve una partición en caso de que el tamaño solicitado sea mayor que 0.

__a) (2 puntos)__ Implementa la función `cut` mediante recursividad final, haciendo uso de las funciones de la librería estándar de Scala `take` y `drop` (descritas en el anexo del examen).

In [ ]:
// INTRODUCE TU RESPUESTA



In [ ]:
run(TestCut(cut))

__b) (2 puntos)__ Implementa la función `cut` mediante la siguiente función de orden superior `iterate`. Esta función generaliza el patrón de diseño iterativo de la función `foldLeft` en dos aspectos: 
* En primer lugar, la actualización de la solución parcial se puede llevar a cabo teniendo en cuenta un prefijo de elementos de la lista, no solamente un elemento.
* En segundo lugar, la iteración no se estructura en tantos pasos como elementos tenga la lista, sino en función de los distintos prefijos calculados.

En la signatura que se muestra a continuación, la función `split` se encarga de dividir la lista de elementos restantes en dos partes: el prefijo que se tendrá en cuenta para actualizar la solución parcial (mediante la función `update`), y el sufijo que se tendrá en cuenta en el siguiente paso de la iteración. 

In [ ]:
def iterate[A, B](l: List[A])(ini: B)(split: List[A] => (List[A], List[A]), update: (B, List[A]) => B): B = 
    l match 
        case Nil => ini
        case t => 
            val (prefix, suffix) = split(t)
            iterate(suffix)(update(ini, prefix))(split, update)

In [ ]:
// INTRODUCE TU RESPUESTA



In [ ]:
run(TestCut(cut))

# Ejercicio 2
__(2 puntos)__

La función `vs` permite particionar un string en función de las distintas ocurrencias de un determinado carácter. Por ejemplo: 

In [ ]:
class TestVs(vs: Char => String => List[String]) extends AnyFlatSpec with should.Matchers:

    "vs" should "work" in:
        vs('#')("abcd#efgh") shouldBe List("abcd", "efgh")
        vs('#')("ab#cd#ef#gh") shouldBe List("ab", "cd", "ef", "gh")
        vs('#')("abcd") shouldBe List("abcd")
        vs('#')("") shouldBe List("")
        vs('#')("##") shouldBe List("", "", "")

Se pide implementar la función `vs` mediante recursividad general (no final). Se recomienda para ello utilizar las siguientes funciones de la librería estándar de Scala, cuyas signaturas se describen en el anexo del enunciado: `indexWhere`, `splitAt` y `drop`.

In [ ]:
// INTRODUCE TU RESPUESTA



In [ ]:
run(TestVs(vs))

# Ejercicio 3
__(4 puntos)__

Los árboles degenerados son árboles binarios cuyos subárboles tienen a lo sumo un hijo no vacío. Un árbol degenerado se puede representar mediante el siguiente tipo de datos algebraico, según el cual, un árbol degenerado puede ser: 
* O bien vacío
* O bien no vacío, en cuyo caso, almacena un valor de tipo `A` y un árbol degenerado (o bien el izquierdo, o bien el derecho, lo cual se representa mediante un valor de tipo `Either`).

In [ ]:
enum TreeDeg[A]: 
    case EmptyDeg()
    case NonEmptyDeg(a: A, child: Either[TreeDeg[A], TreeDeg[A]])

import TreeDeg._

__a) (1.5 puntos)__ Implementa una función que permita transformar la representación de un árbol binario estándar (cuya definición `Tree` se encontrará en el anexo de este enunciado) en la representación de árboles degenerados `TreeDeg` definida más arriba. En caso de que el árbol binario de entrada no sea degenerado, la transformación no se llevará a cabo, tal y como se indica en los siguientes tests:  

In [ ]:
class TestToDegenerate(toDeg: Tree[Int] => Option[TreeDeg[Int]]) extends AnyFlatSpec with should.Matchers:

    "toDeg" should "work" in:
        toDeg(void) shouldBe Some(EmptyDeg())
        toDeg(leaf(1)) shouldBe Some(NonEmptyDeg(1, Left(EmptyDeg())))
        toDeg(left(leaf(1), 2)) shouldBe Some(NonEmptyDeg(2, Left(NonEmptyDeg(1, Left(EmptyDeg())))))
        toDeg(right(2, leaf(1)))  shouldBe Some(NonEmptyDeg(2, Right(NonEmptyDeg(1, Left(EmptyDeg())))))
        toDeg(left(left(leaf(3), 2), 1)) shouldBe Some(NonEmptyDeg(1, Left(NonEmptyDeg(2, Left(NonEmptyDeg(3, Left(EmptyDeg())))))))
        toDeg(left(right(2, left(right(4, leaf(5)), 3)),1)) shouldBe 
            Some(NonEmptyDeg(1, Left(NonEmptyDeg(2, Right(NonEmptyDeg(3, Left(NonEmptyDeg(4, Right(NonEmptyDeg(5, Left(EmptyDeg())))))))))))
        toDeg(node(leaf(1), 2, leaf(3))) shouldBe None
        toDeg(left(node(leaf(3), 2, leaf(3)), 1)) shouldBe None

Obsérvese que las hojas (árboles no vacíos degenerados con un solo nodo) tienen su único árbol vacío representado en el valor de tipo `Left`. Se podría haber elegido el valor de tipo `Right` como convención alternativa. 

In [ ]:
// INTRODUCE TU RESPUESTA



In [ ]:
run(TestToDegenerate(toDegenerate))

__b) (1.5 puntos)__ Implementa la misma función, pero teniendo en cuenta obligatoriamente la siguiente función de orden superior (equivalente a la función `map` de las listas para el tipo `Option`):

In [ ]:
def mapOption[A, B](o: Option[A])(f: A => B): Option[B] = 
    o match 
        case None => None
        case Some(a) => Some(f(a))

In [ ]:
// INTRODUCE TU RESPUESTA



In [ ]:
run(TestToDegenerate(toDegenerate))

__c) (0.5 puntos)__ Implementa la función inversa, que dado un árbol degenerado en la representación `TreeDeg` devuelve un árbol en la representación `Tree`. 

In [ ]:
class TestFromDegenerate(fromDeg: TreeDeg[Int] => Tree[Int]) extends AnyFlatSpec with should.Matchers:

    "fromDeg" should "work" in:
        fromDeg(EmptyDeg()) shouldBe void
        fromDeg(NonEmptyDeg(1, Left(EmptyDeg()))) shouldBe leaf(1)
        fromDeg(NonEmptyDeg(2, Left(NonEmptyDeg(1, Left(EmptyDeg()))))) shouldBe left(leaf(1), 2)
        fromDeg(NonEmptyDeg(2, Right(NonEmptyDeg(1, Left(EmptyDeg())))))  shouldBe right(2, leaf(1))
        fromDeg(NonEmptyDeg(1, Left(NonEmptyDeg(2, Left(NonEmptyDeg(3, Left(EmptyDeg()))))))) shouldBe left(left(leaf(3), 2), 1)
        fromDeg(NonEmptyDeg(1, Left(NonEmptyDeg(2, Right(NonEmptyDeg(3, Left(NonEmptyDeg(4, Right(NonEmptyDeg(5, Left(EmptyDeg()))))))))))) shouldBe 
            left(right(2, left(right(4, leaf(5)), 3)),1)

In [ ]:
// INTRODUCE TU RESPUESTA



In [ ]:
run(TestFromDegenerate(fromDegenerate))

__d) (0.5 puntos)__ Implementa la siguiente ley: dado un árbol binario degenerado, la transformación a la representación `TreeDeg` mediante la función `toDegenerate` y la transformación de dicho resultado de vuelta a la representación `Tree`, es la identidad (es decir, el resultado es el árbol binario de partida).

In [ ]:
// INTRODUCE TU RESPUESTA

def law[A](t: Tree[A]): Boolean = 
    ???